# 🍫 [Day 19](https://adventofcode.com/2020/day/19)

In [1]:
import re 
from collections import defaultdict

def do_you_like_regexps(rules, part1=True, verbose=False):
    # First determine the orders for the rules
    num_rules = len(rules)
    sorted_rules = {}
    depends_on = {}
    unlocks = defaultdict(lambda: [])
    
    roots = []
    for line in rules:
        # Parse
        rule, aux = line.split(': ', 1)
        rule = int(rule)
        aux = [int(x) for x in aux.replace('|', '').split() if x.isnumeric()]
        sorted_rules[rule] = line
        
        # Base case
        if len(aux) == 0:
            roots.append(rule)
            continue
        
        # Store graph
        depends_on[rule] = aux
        for r in list(set(aux)):
            unlocks[r].append(rule)
            
                
    # Then, resolve each rule in the correct order
    regexps = defaultdict(lambda: None)
    queue = roots
    while len(queue):
        rule_id = queue.pop()
        # Resolve regexp
        rule, aux = sorted_rules[rule_id].split(': ', 1)
        rule = int(rule)
        if aux.startswith('"'):
            regexps[rule] = aux.replace('"', '')
        # Special case: rule 8 and rule 11
        elif not part1 and rule_id == 8:
            # 8: 42 | 42 8 
            regexps[8] = f'({regexps[42]})+'
        elif not part1 and rule_id == 11:
            # 11: 42 31 | 42 11 31
            # Hacky, to handle same number of repeats
            pattern = []
            for k in range(1, 6):
                pattern.append(f'{regexps[42] * k + regexps[31] * k}')
            regexps[11] = '|'.join(pattern)
            regexps[11] = f'({regexps[11]})'
        else:
            pattern = ''
            acc = 0
            pipe = False
            for c in aux:
                if c == ' ':
                    if acc != 0:
                        pattern = f'{pattern}{regexps[acc]}'
                        acc = 0
                elif c == '|':
                    pipe = True
                    pattern = f'{pattern}|'
                else:
                    acc = acc * 10 + int(c)
            pattern = f'{pattern}{regexps[acc]}'
            if pipe:
                pattern = f'({pattern})'
            regexps[rule] = pattern
        if verbose:
            print(f"Rule {rule} matches pattern {regexps[rule]}")
        
        # Add next children
        for r in unlocks[rule_id]:
            if sum(regexps[x] is None for x in depends_on[r]) == 0:
                queue.append(r)
    # Return
    return regexps


def check_messages(rules, messages, part1=True):
    pattern = do_you_like_regexps(rules, part1=part1)[0]
    pattern = re.compile(f'^{pattern}$')
    return sum(int(pattern.match(m) is not None) for m in messages)

In [2]:
with open('inputs/day19.txt', 'r') as f:
    inputs = f.read()
    rules, messages = inputs.split('\n\n')
    rules = rules.splitlines()
    messages = messages.splitlines()

print(f"There are {check_messages(rules, messages)} messages that match rule 0.")
print(f"There are {check_messages(rules, messages, False)} messages that match rule 0 with recursive rules.")

There are 241 messages that match rule 0.
There are 424 messages that match rule 0 with recursive rules.
